<a href="https://colab.research.google.com/github/bcgsc/pori_graphkb_python/blob/feature%2Fjupyter/docs/pori_graphkb_python_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GraphKB Variant Matching Tutorial

This tutorial is an interactive notebook which can be run using ([google colab](https://colab.research.google.com/drive/1JnbK3Tm9f9XhuwUqvmbXReLGk_2GuaI4?usp=sharing)) or a local jupyter server (**reccommended** if matching patient data). This tutorial will cover basic matching of variants using the python GraphKB adaptor against an instance of the GraphKB API. 

Users must first have login credentials to an instance of GraphKB API (or use the demo server). Note for users using the demo credentials and server, the data is limited and more complete annotations would be expected for a production instance of GraphKB.

For the purposes of this tutorial we will be matching the known KRAS variant `p.G12D` to the demo instance of GraphKB. You can adjust the API instance by changing the setup variables below

To run this locally download this file and start the server from the command line as follows

```
jupyter notebook notebook.ipynb
```

You should now be able to see the notebook by openining `http://localhost:8888` in your browser

In [39]:
!pip3 install graphkb

In [40]:
from getpass import getpass

from graphkb import GraphKBConnection

GKB_API_URL = 'https://pori-demo.bcgsc.ca/graphkb-api/api'
GKB_USER = 'colab_demo'
GKB_PASSWORD = 'colab_demo'

graphkb_conn = GraphKBConnection(GKB_API_URL, False)
graphkb_conn.login(GKB_USER, GKB_PASSWORD)

## Matching Variants

Now you are ready to match variants

In [41]:
from graphkb.match import match_positional_variant

variant_name = 'KRAS:p.G12D'
variant_matches = match_positional_variant(graphkb_conn, variant_name)

print(f'{variant_name} matched {len(variant_matches)} other variant representations')
print()

for match in variant_matches:
    print(variant_name, 'will match', match['displayName'])


KRAS:p.G12D matched 7 other variant representations

KRAS:p.G12D will match KRAS:p.(G12_G13)mut
KRAS:p.G12D will match KRAS:p.G12mut
KRAS:p.G12D will match KRAS:p.G12D
KRAS:p.G12D will match chr12:g.25398284C>T
KRAS:p.G12D will match KRAS:p.G12
KRAS:p.G12D will match KRAS:p.?12mut
KRAS:p.G12D will match KRAS mutation


We can see above that the KRAS protein variant has been matched to a number of other less specific mentions (ex. KRAS:p.G12mut) and also genomic equivalents (chr12:g.25398284C>T). Note that the results here will be dependent on the instance of GraphKB you are accessing.

## Annotating Variants

Now that we have matched the variant we will fetch the related statements to annotate this variant with its possible relevance

In [42]:
from graphkb.constants import BASE_RETURN_PROPERTIES, GENERIC_RETURN_PROPERTIES
from graphkb.util import convert_to_rid_list

# return properties should be customized to the users needs
return_props = (
    BASE_RETURN_PROPERTIES
    + ['sourceId', 'source.name', 'source.displayName']
    + [f'conditions.{p}' for p in GENERIC_RETURN_PROPERTIES]
    + [f'subject.{p}' for p in GENERIC_RETURN_PROPERTIES]
    + [f'evidence.{p}' for p in GENERIC_RETURN_PROPERTIES]
    + [f'relevance.{p}' for p in GENERIC_RETURN_PROPERTIES]
    + [f'evidenceLevel.{p}' for p in GENERIC_RETURN_PROPERTIES]
)

statements = graphkb_conn.query(
    {
        'target': 'Statement',
        'filters': {'conditions': convert_to_rid_list(variant_matches), 'operator': 'CONTAINSANY'},
        'returnProperties': return_props,
    }
)
print(f'annotated {len(variant_matches)} variant matches with {len(statements)} statements')
print()

for statement in statements[:5]:
    print(
        [c['displayName'] for c in statement['conditions'] if c['@class'].endswith('Variant')],
        statement['relevance']['displayName'],
        statement['subject']['displayName'],
        statement['source']['displayName'] if statement['source'] else '',
        [c['displayName'] for c in statement['evidence']],
    )

annotated 7 variant matches with 96 statements

['KRAS:p.(G12_G13)mut'] resistance Gefitinib [c1855] CIViC ['pmid:15696205']
['KRAS:p.(G12_G13)mut'] resistance Panitumumab [c1857] CIViC ['pmid:19223544']
['KRAS:p.(G12_G13)mut'] resistance Cetuximab [c1723] CIViC ['pmid:19223544']
['KRAS:p.(G12_G13)mut'] resistance Cetuximab [c1723] CIViC ['pmid:19603024']
['KRAS:p.(G12_G13)mut'] resistance Panitumumab [c1857] CIViC ['pmid:18316791']
